# Alcholic Anonymous News articles extraction from Tribune Newspaper

## Importing the necessary Libraries

In [1]:
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver # python web automation driver
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

## Lists for Alcholic Anonymous News Articles extracted data

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, image_urls = [], [], [], [], [], [], [], [], []

## Scraping Google Search Results urls related to Tribune Newspaper

In [3]:
keyword = 'Alcoholic Anonymous site:www.tribuneindia.com'

url = 'https://www.google.com/search?q=' + '+'.join(keyword.split())

options = Options()
options.add_argument("--headless")
browser = webdriver.Chrome(options=options)
browser.get(url)

page = browser.page_source
soup = BeautifulSoup(page, 'lxml')
linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
urls.extend(linky)

## Finding the total no.of.pages from Google Search Results and passing the value through the while loop

In [4]:
max_pages = int(int(soup.select_one('#resultStats').text.split(' ')[1])//10)
time.sleep(1)
index = 1

while True:
    index +=1
    if index == max_pages:
        page = browser.page_source
        soup = BeautifulSoup(page, 'lxml')
        linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
        urls.extend(linky)
        break
    browser.find_element_by_xpath('//*[@id="pnnext"]/span[2]').click()
    time.sleep(2)
    page = browser.page_source
    soup = BeautifulSoup(page, 'lxml')
    linky = [soup.select('.r')[i].a['href'] for i in range(len(soup.select('.r')))]
    urls.extend(linky)
    sys.stdout.write('\r' + str(index) + ' : ' + str(max_pages) + '\r')
    sys.stdout.flush()
    
browser.quit()

## In google search search results it store cache url which was in the same tag. To remove duplicates urls entries in the list by executing below line

In [ ]:
urls = list(dict.fromkeys(urls))
len(urls)

100

## Scraping the Articles details from Tribune Newspaper with the help of Newspaper

In [ ]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        article = Article(url, timeout=4)
        article.download()
        article.parse()

        try:
            # Extracts the Headlines 
            headlines.append(article.title)
        except:
            headlines.append(None)
            
        try:
            # Extracts the Descriptions
            descriptions.append(article.meta_description)
        except:
            descriptions.append(None)
            
        try:
            # Extracts the Authors
            authors.append(article.authors)
        except:
            authors.append(None)
            
        try:
            # Extracts the published dates
            dates.append(str(article.publish_date))
        except:
            dates.append(None)

        try:
            # Extracts the news articles
            news.append(article.text)
        except:
            news.append(None)
            
        try:
            # Extracts Keywords and Summaries
            article.nlp()
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
                    
        try:
            # Extracts the image urls
            image_urls.append(article.top_image)
        except:
            image_urls.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)
        image_urls.append(None)
        

    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

## Checking Array Length of each list to create DataFrame

In [ ]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls), len(image_urls))

## Creating a csv file after checking array length

In [ ]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,
                    'Source_URLs' : urls,
                    'Image_URLs' : image_urls})

tbl.to_csv('Tribune.csv', index=False)